In [1]:
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge,Lasso, LogisticRegression,SGDClassifier,SGDRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor,ExtraTreesClassifier,ExtraTreesRegressor
from sklearn.ensemble import HistGradientBoostingClassifier,HistGradientBoostingRegressor
from xgboost import XGBRegressor,XGBClassifier
from lightgbm import LGBMRegressor,LGBMClassifier

- 분류
  - KNeighborsClassifier
  - LogisticRegression
  - DecisionTreeClassifier
  - SGDClassifier
  - RandomForestClassifier
  - ExtraTreesClassifier
  - HistGradientBoostingClassifier
  - XGBClassifier, LGBMClassifier

  
- 회귀
  - KNeighborsRegressor
  - LinearRegression
  - Ridge,Lasso  
  - DecisionTreeRegressor
  - SGDRegressor
  - RandomForestRegressor
  - ExtraTreesRegressor
  - HistGradientBoostingRegressor  
  - XGBRegressor, LGBMClassifier
- 회귀중에서 성능을 높이기위한 행동
  - 선형 --> 비선형 - poly

- 규제하기전에는 반드시 표준화
- 대략적인 성능측정은 교차검증을통해서
- 파라메터 최적화 - GridSearch, Random.....
- 범주형 데이터는 라벨인코더 또는 원핫인코더  
- 피처엔지니어링
  - 날자가 포함된 데이터는 특성상. .년 월 일로 구분하고
  - 공휴일이나 휴일은 중요한 지표가 된다.

1. 데이터 분석 및 전처리
2. 각 모델에 대한 딕셔너리
- 파이프라인을 구축(1,2번) : 최적의 모델 선택
- 하이퍼 파라메터 튜닝을 통해 최상의 모델 선택

In [ ]:
# !wget -O data.zip 'https://drive.google.com/uc?export=download&id=1xbKFnrb8kHlMCROF_lk_0plM8K9EpMGN'

In [ ]:
# !unzip -O cp949 /content/data.zip

In [ ]:
# 공휴일
# !pip install workalendar

In [ ]:
import pandas as pd
import numpy as np
from workalendar.asia import SouthKorea

In [ ]:
df = pd.read_csv('/content/교통사고피해에측/open/train.csv')
df.head(2)
# ECLO 인명피해 심각도 사망자수*10 + 중상자수*5 + 경상자수*3 + 부상자수*1
# 시각관 공간 정보를 이용해서 ECLO 예측 모델 개수
sum(['대구광역시' not in city for city in df['시군구'].value_counts().index])

0

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
# 전처리 - 결측치 처리
#  ['가해운전자 연령'] == '미분류' 드랍
df['가해운전자 연령'] = df['가해운전자 연령'].apply(lambda x : np.nan if x == '미분류' else x)
df = df.dropna()
df['가해운전자 연령'] = df['가해운전자 연령'].apply(lambda x : int(x[: x.find('세')]) )
df['피해운전자 연령'] = df['피해운전자 연령'].apply(lambda x : np.nan if x == '미분류' else x)
df = df.dropna()
df['피해운전자 연령'] = df['피해운전자 연령'].apply(lambda x : int(x[: x.find('세')]) )

<ipython-input-8-73bffef756cd>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['가해운전자 연령'] = df['가해운전자 연령'].apply(lambda x : int(x[: x.find('세')]) )
<ipython-input-8-73bffef756cd>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['피해운전자 연령'] = df['피해운전자 연령'].apply(lambda x : np.nan if x == '미분류' else x)


In [ ]:
# 주소를 시 군 구 로 분할해서 df['city'] df['gun'] df['gu']
# 사고일시 년 월
# 범주형 데이터는 수치로...일단 라벨 인코더
# X, y 분류
# Q 전부 다 써야 하나? 아니면 ... 골라야하나... 고른다면 그 기준은?

In [ ]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [ ]:
X.shape

(37567, 22)

In [ ]:
X=X.dropna()

In [ ]:
X.shape

(37567, 22)

In [ ]:
# 손실된 데이터 양
1-(38618 / 39609)

0.025019566260193415

In [ ]:
# 주소를 시/군/구로 분할
df['city'] = df['시군구'].apply(lambda x: x.split()[0])
df['gun'] = df['시군구'].apply(lambda x: x.split()[1])
df['gu'] = df['시군구'].apply(lambda x: x.split()[2])

In [ ]:
# 날짜
X['year'] = pd.to_datetime(X['사고일시']).dt.year
X['month'] = pd.to_datetime(X['사고일시']).dt.month
X['day'] = pd.to_datetime(X['사고일시']).dt.day

In [ ]:
# 평일은 1, 주말/공휴일 0   문자열 yyyy-mm-dd 형태가필요
X['사고일시'] = pd.to_datetime(X['사고일시'])
X['workingday'] =  X['사고일시'].apply(lambda x : 1 if SouthKorea().is_working_day(x) else 0)

In [ ]:
X.head(1)
X = X.drop(columns=['ID','사고일시','요일','시군구'])

In [ ]:
# X 에서 범주형 데이터 추출
X.describe(include='O').columns

Index(['기상상태', '도로형태', '노면상태', '사고유형', '사고유형 - 세부분류', '법규위반', '가해운전자 차종',
       '가해운전자 성별', '가해운전자 상해정도', '피해운전자 차종', '피해운전자 성별', '피해운전자 상해정도'],
      dtype='object')

In [ ]:
X.describe().columns

Index(['가해운전자 연령', '피해운전자 연령', '사망자수', '중상자수', '경상자수', '부상자수', 'year', 'month',
       'day', 'workingday'],
      dtype='object')

In [ ]:
for col in X.describe(include='O').columns:
  le = LabelEncoder()
  X[col] =  le.fit_transform(X[col])

In [ ]:
X.head()

,기상상태,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,...,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,year,month,day,workingday
0,2,6,0,0,1,7,5,1,51,4,...,70,5,0,1,0,0,2019,1,1,0
1,5,6,0,0,2,2,5,0,39,4,...,61,0,0,0,1,0,2019,1,1,0
2,2,6,0,0,4,7,5,0,70,4,...,38,0,0,0,1,0,2019,1,1,0
3,2,6,0,1,5,7,5,0,49,4,...,36,5,0,1,0,0,2019,1,1,0
4,2,6,0,1,5,7,5,0,30,4,...,52,0,0,0,1,0,2019,1,1,0


기본 데이터 탐색 끝

In [ ]:
# 표준화
from sklearn.preprocessing import StandardScaler
for col in ['가해운전자 연령','피해운전자 연령','사망자수','중상자수','경상자수','부상자수']:
  ss = StandardScaler()
  X[col] = ss.fit_transform(X[col].to_numpy().reshape(-1,1))

In [ ]:
X

,기상상태,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,...,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,year,month,day,workingday
0,2,6,0,0,1,7,5,1,0.191845,4,...,1.494463,5,-0.082075,1.461580,-1.099999,-0.301423,2019,1,1,0
1,5,6,0,0,2,2,5,0,-0.557101,4,...,0.961868,0,-0.082075,-0.525794,-0.097606,-0.301423,2019,1,1,0
2,2,6,0,0,4,7,5,0,1.377677,4,...,-0.399208,0,-0.082075,-0.525794,-0.097606,-0.301423,2019,1,1,0
3,2,6,0,1,5,7,5,0,0.067021,4,...,-0.517562,5,-0.082075,1.461580,-1.099999,-0.301423,2019,1,1,0
4,2,6,0,1,5,7,5,0,-1.118811,4,...,0.429273,0,-0.082075,-0.525794,-0.097606,-0.301423,2019,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,2,1,0,1,6,5,5,1,0.254257,4,...,-0.990980,0,-0.082075,-0.525794,-0.097606,-0.301423,2021,12,31,1
39605,2,6,0,1,6,6,5,1,0.753555,4,...,0.429273,0,-0.082075,-0.525794,-0.097606,-0.301423,2021,12,31,1
39606,2,1,0,1,6,1,5,0,0.753555,5,...,1.671995,5,-0.082075,3.448953,-1.099999,-0.301423,2021,12,31,1
39607,2,3,0,1,5,7,5,0,-0.494689,4,...,0.725159,0,-0.082075,-0.525794,-0.097606,-0.301423,2021,12,31,1


In [ ]:
# # 각 열을 반복하여 y 데이터프레임의 열을 인덱싱
# y_encoded = pd.DataFrame()
# label_encoders = []

# for col in y.columns:
#     le = LabelEncoder()
#     encoded_col = le.fit_transform(y[col])
#     y_encoded[col] = encoded_col
#     label_encoders.append(le)

In [ ]:
# 회귀
regression_lists = [KNeighborsRegressor(),
LinearRegression(),
Ridge,Lasso(),
DecisionTreeRegressor(),
SGDRegressor(),
RandomForestRegressor(),
ExtraTreesRegressor(),
HistGradientBoostingRegressor(),
XGBRegressor(), LGBMClassifier(),]

In [ ]:
from tqdm import tqdm
from sklearn.model_selection import cross_validate, KFold
cv = KFold(n_splits=5, shuffle=True,random_state=42)

train_score_list, test_score_list = [],[]
for reg in tqdm(regression_lists):
  scores = cross_validate( reg,X,y, return_train_score=True, n_jobs=-1,cv=cv)
  train_score_list.append( np.mean(scores['train_score']) )
  test_score_list.append( np.mean(scores['test_score']) )

100%|██████████| 1/1 [00:30<00:00, 30.15s/it]


In [ ]:
y

0         5
1         3
2         3
3         5
4         3
         ..
39604     3
39605     3
39606    10
39607     3
39608     6
Name: ECLO, Length: 37567, dtype: int64

In [ ]:
X.describe(include='all')

,기상상태,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,...,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,year,month,day,workingday
count,37567.000000,37567.000000,37567.000000,37567.000000,37567.000000,37567.000000,37567.000000,37567.000000,3.756700e+04,37567.000000,...,3.756700e+04,37567.000000,3.756700e+04,3.756700e+04,3.756700e+04,3.756700e+04,37567.000000,37567.000000,37567.000000,37567.000000
mean,2.117071,3.557271,0.308782,0.829372,4.203902,6.043203,6.065829,0.251258,1.059185e-17,3.376660,...,-6.052484e-18,1.487556,1.702261e-17,2.874930e-17,3.073527e-17,-7.036013e-17,2019.940001,6.676604,15.820747,0.734049
std,0.471493,2.631246,1.065073,0.376189,2.644212,2.027948,2.130894,0.433742,1.000013e+00,1.371463,...,1.000013e+00,2.129017,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,0.818723,3.406442,8.791668,0.441844
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.679117e+00,0.000000,...,-2.588765e+00,0.000000,-8.207492e-02,-5.257941e-01,-1.099999e+00,-3.014233e-01,2019.000000,1.000000,1.000000,0.000000
25%,2.000000,1.000000,0.000000,1.000000,0.000000,6.000000,5.000000,0.000000,-8.067503e-01,4.000000,...,-8.134485e-01,0.000000,-8.207492e-02,-5.257941e-01,-9.760576e-02,-3.014233e-01,2019.000000,4.000000,8.000000,0.000000
50%,2.000000,6.000000,0.000000,1.000000,6.000000,7.000000,5.000000,0.000000,1.294330e-01,4.000000,...,1.503256e-02,0.000000,-8.207492e-02,-5.257941e-01,-9.760576e-02,-3.014233e-01,2020.000000,7.000000,16.000000,1.000000
75%,2.000000,6.000000,0.000000,1.000000,6.000000,7.000000,7.000000,1.000000,7.535552e-01,4.000000,...,7.843364e-01,4.000000,-8.207492e-02,-5.257941e-01,-9.760576e-02,-3.014233e-01,2021.000000,10.000000,23.000000,1.000000
max,5.000000,10.000000,5.000000,1.000000,8.000000,10.000000,11.000000,1.000000,3.125220e+00,5.000000,...,3.151425e+00,5.000000,2.354481e+01,1.139845e+01,2.095266e+01,2.562843e+01,2021.000000,12.000000,31.000000,1.000000


In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_validate
from tqdm import tqdm

# KNeighborsRegressor의 인스턴스 생성
regressor = KNeighborsRegressor(n_neighbors=5)

# 다른 추정기 객체들도 필요한 인스턴스를 생성합니다.
# regression_lists 리스트에 여러 추정기 객체를 추가합니다.
regression_lists = [regressor]

# 교차 검증을 수행합니다.
train_score_list, test_score_list = [], []

for reg in tqdm(regression_lists):
    # cross_validate 함수에 추정기 객체 `reg`를 전달합니다.
    scores = cross_validate(reg, X, y, return_train_score=True, n_jobs=-1, cv=cv)
    train_score_list.append(np.mean(scores['train_score']))
    test_score_list.append(np.mean(scores['test_score']))

100%|██████████| 1/1 [00:34<00:00, 34.27s/it]


In [ ]:
train_s = [round(data,3) for data in train_score_list]
test_s = [round(data,3) for data in test_score_list]
train_s[5] = 0
test_s[5] = 0
pd.DataFrame(zip(train_s,test_s),columns=['train_score','test_score'])

In [ ]:
print(train_s)

파이프라인
  - 데이터수집, 데이터 처리, 데이터 분석/모델링 -
   검증

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
df = pd.read_csv('교통사고피해에측/open/train.csv')
target = 'ECLO'
# 특성선택(필요없는 열 제외)
featrue = df.drop(columns=['ID','ECLO','요일'])

# 데이터 분할
X_train, X_test, y_train, y_test =  train_test_split(featrue, df[target], test_size=0.2, random_state=42)

# 범주형 및 수치형 컬럼 구분
# [col for col in featrue.columns if featrue[col].dtype == 'object']
# [col for col in featrue.columns if featrue[col].dtype in ['int64','float64'] ]
categorical_cols = featrue.describe(include='O').columns.to_numpy()
numerical_cols = featrue.describe().columns.to_numpy()

# 데이터 전처리 파이프라인 - 수치형은 평균 범주형은 최빈값
numerical_transformer = Pipeline([
  ('imputer',SimpleImputer(strategy = 'mean')),
  ('scaler',StandardScaler())
])

categorical_transformer = Pipeline([
 ('imputer',SimpleImputer(strategy = 'most_frequent')),  # 범주형 데이터 결측치 처리
 ('onehot', OneHotEncoder(handle_unknown='ignore')) # 모르는 범주형 데이터가 나오면 어떠한 특성도 부여하지 않도록 전부 0으로채움
])
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers = [
        ('num',numerical_transformer,numerical_cols),
        ('cat',categorical_transformer,categorical_cols)
    ]
)
# 모델 파이프 라인 구축
model = Pipeline([
    ('proprocessor',preprocessor),
    ('regressor', XGBRegressor())
])

In [ ]:
model.fit(X_train,y_train)
model.score(X_train,y_train), model.score(X_test,y_test)